<a href="https://colab.research.google.com/github/jshogland/SpatialModelingTutorials/blob/main/Notebooks/ShawneeNF_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Percent Tree Canopy Cover Notebook
## In this notebook we will explore how to build a simple random sample of point locations, extract remotely sensed data for those locations, and use that data to estimate % Tree Cover for the Shawnee National Forest.

### Project objective:
- Build a machine learning model from estimates of 2016 percent forest canopy cover, elevation data, and Landsat 8 imagery for that same time period
- Apply that model to elevation and Landsat 8 data acquired in 2021 to estimate percent forest canopy cover in 2021
- Compare our model estimates to [MLRC](https://www.mrlc.gov/data-services-page) % forest canopy cover estimates for the year 2021

#### Notebook Sections:
1. Setup
2. Create a Sample
3. Download data
4. Creating the Data Frame
5. Clean data
6. Preprocessing
7. Build a predictive model
8. Estimate Percent Tree Canopy Cover

#### Data sources
- [Landsat 8](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2) - Extracted from Planetary Computer ([STAC](https://stacspec.org/en))
- [Tree Canopy Cover 2016](https://apps.fs.usda.gov/fsgisx01/rest/services/RDW_LandscapeAndWildlife/NLCD_2016_TreeCanopyCover_CONUS/ImageServer) - Extracted from Landfire Image Service ([REST](https://developers.arcgis.com/rest/services-reference/enterprise/image-service/))
- [USGS Elevation](https://www.usgs.gov/3d-elevation-program) - Extracted from USGS 3DEP program ([py3dep](https://github.com/hyriver/py3dep))
- [National Forest Boundary](https://www.openstreetmap.org/#map=9/34.306/-112.242) - Extracted from Open Street Maps ([osmnx](https://osmnx.readthedocs.io/en/stable/))

Author John Hogland 10/1/2024

### Section 1: Setup
#### Installing software for Colab

In [ ]:
!pip install mapclassify
!pip install osmnx
!pip install raster_tools
!pip install planetary-computer
!pip install pystac-client
!pip install stackstac
!pip install py3dep

#### Importing packages

In [ ]:
#get packages
import osmnx as ox, planetary_computer, pystac_client, stackstac
import geopandas as gpd, pandas as pd, os, numpy as np, requests, urllib, py3dep

from raster_tools import Raster,zonal, general, Vector
from shapely import geometry

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

### Section 2: Create a sample
#### In this section we will extract the spatial boundary of the Shawnee National Forest using Open Street Maps (OSM) Web Service and create a simple random sample of locations within the boundary of the National forest.

#### Key learning points:
- Using OSM to get vector data (downloading)
- Generating a simple random sample of point location that will be used to create a data frame of response and predictor variables
- How to Visualize the the boundary and point locations

#### Coding Steps:
1. Use osmnx and OSM to get a GeoDataFrame (polygons) of the boundary of the Shawnee National Forest
2. Use the Shawnee National Forest boundary and Geopanda's sample_points function to create a simple random sample of point locations
3. Create a interactive map that displays our point locations

##### Step 1: Get the Shawnee National Forest boundary

In [ ]:
#use osmnx OpenStreetMaps to get the boundary of the NF (GeoDataFrame)
nf=ox.geocode_to_gdf('Shawnee National Forest, IL, USA')

#Look at the coordinate system of the GeoDataFrame
print('Original CRS =',nf.crs)

#project the GeoDataFrame to Albers equal area EPSG:5070
nfp=nf.to_crs(5070)

#Look at the GeoDataFrame
display(nfp)

#Plot the national forest
nfp.plot(figsize=(15,15))

#Why are we projecting to Albers equal Area?

#How many acres is the National Forest?
print('Number of acres =',(nfp.area * 0.000247105).values[0])

#How many polygons are in the National Forest?
print('Number of multipolygons =',nfp.shape[0])
print('Number of polygons =',nfp.explode().shape[0])



##### Step 3: Create the simple random sample

In [ ]:
#us random sample function to create 2,000 locations within the nfp
mpnts=nfp.sample_points(2000)

#Look at the GeoSeries
mpnts

#Why 2,000 locations? Why not 100 or 10.000?

#How many records do we have?
#print('Number of records =',mpnts.shape[0])

#How many observations do we have?
#print('Number of points (observations) =',mpnts.explode().shape[0])

#How can we convert our multipoint to points?
pnts=mpnts.explode().reset_index(drop=True) #reseting the index to clean things up

##### Step 4: Visualize the national forest boundary and point locations

In [ ]:
#Use geopandas explore function to create a interactive map
m=nfp.explore(color='blue') #create the map using the National Forest Boundary
m=pnts.explore(m=m,color='yellow') #add out points to the map
m #show the map

#Can we save our map to share with others?
#m.save(sample.html)

#How can we plot our map?
# p=nfp.plot(edgecolor='blue',facecolor='none',figsize=(15,15))
# p=pnts.plot(ax=p,color='yellow')
# p


### Section 3: Downloading Raster Data
#### In this section we will focus on download raster dataset from various sources.

#### Key learning points:
- Accessing data from different web services
- Deciding where to do your processing (client or server side)
- How and when to store the data locally
- How to visualizing the raster surfaces

#### Coding Steps:
1. Create functions to download data from STAC, REST, WCS
2. Use those functions to create Raster datasets
3. Visualize Your Raster datasets


##### Step 1: Creating download definitions

In [ ]:
from owslib.wcs import WebCoverageService
#create definition to mosaic stac data
def mosaic_stac(xr):
    '''
    Creates a mosaic from multi-temporal xarray stack

    xr=Xarray object

    returns xarray mosaic
    '''
    return stackstac.mosaic(xr)

#create definition to extract stac data
def get_stac_data(ply,url,name,bands,res=30,crs=5070,
                  out_prefix='ls8',**kwarg):
    '''
    gets tiled data from planetary computer as a dask backed xarray that intersects the geometry of the point, line, or polygon

    ply = (geoseries or geodataframe) of the study area
    url = (string) base url to planetary computer https://planetarycomputer.microsoft.com/api/stac/v1
    name = (string) catelog resource e.g., "sentinel-2-l2a"
    bands = (list(string)) of attributes ['red', 'blue', 'green', 'nir08', 'lwir11','swir16', 'swir22']
    qry =  (dictionary) of property values {'eo:cloud_cover':{'lt':1}}
    res = (tuple of numbers) output resolution (x,y)
    crs = (int) output crs
    dt = (string) data time intervale e.g., one month: 2023-06, range: 2023-06-02/2023-06-17
    limit = (int) max number of items to return
    out_prefix = (string) prefix used to save the image

    returns a Raster object
    '''
    geo = ply.to_crs('4326').envelope.geometry[0]
    xmin,ymin,xmax,ymax=ply.total_bounds
    catalog = pystac_client.Client.open(url, modifier=planetary_computer.sign_inplace)
    srch = catalog.search(collections=name, intersects=geo, **kwarg)
    ic = srch.item_collection()
    if(len(ic.items)>0):
        xra = stackstac.stack(ic,resolution=res,epsg=crs)
        xra = mosaic_stac(xra)
        rs=Raster(xra.sel(band=bands,x=slice(xmin,xmax),y=slice(ymax,ymin)))
        outpath=out_prefix+'.tif'
        rs.save(outpath)
        rs=Raster(outpath)
    else:
        rs=None

    return rs

#Create definition to extract image service data
def get_image_service_data(url, ply, out_prefix,res=30,outSR=""):
    '''
    extracts a list of images from a image service given a url, polygon, and output prefix name

    url = (string) path to image service e.g., url=r'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF230/US_230EVT/ImageServer'
    ply = (geoseries or geodataframe) of the study area
    out_prefix = (string) prefix used to save each image

    returns a list of Raster objects, one for each tile
    '''
    layerInfo=requests.get(url+'?f=pjson')
    dic=layerInfo.json()
    #print(dic)
    spr=dic['spatialReference']
    m_width=dic['maxImageWidth']
    m_height=dic['maxImageHeight']
    fitem=next(iter(spr))
    ply2=ply.to_crs(spr[fitem])

    xmin,ymin,xmax,ymax=ply2.total_bounds

    wcells=int((xmax-xmin)/res)
    hcells=int((ymax-ymin)/res)

    if(wcells<m_width):
        m_width=wcells

    if(hcells<m_height):
        m_height=hcells


    wcells_l=np.arange(0,wcells,m_width)
    hcells_l=np.arange(0,hcells,m_height)

    xmax2=xmin
    ymax2=ymin

    tile=1

    rs_lst=[]
    for w in wcells_l:
        for h in hcells_l:
            xmax2 = (m_width*res+xmax2)
            ymax2 = (m_height*res+ymax2)

            qry = url+'/exportImage?'
            parm = {
                'f':'json',
                'bbox':','.join([str(xmin),str(ymin),str(xmax2),str(ymax2)]),
                'size':str(m_width) + ',' + str(m_height),
                'imageSR':outSR,
                'format':'tiff'
            }
            #print(parm['bbox'])
            response=requests.get(qry,parm)
            if response.status_code == 200:
                img_url=response.json()['href']
                outname=out_prefix + str(tile) + '.tif'
                urllib.request.urlretrieve(img_url, outname)
                rs_lst.append(Raster(outname))
                tile+=1

    return rs_lst

# Creat definition for WCS download
def get_wcs_data(url,ply,service_name='mrlc_download__nlcd_tcc_conus_2021_v2021-4',out_prefix = 'tcc'):
    '''
    Extracts saves an image from a WCS given url, polygon boundary, and service name. Images are saved in the same location as the notebook.
    url = (string) path to wcs e.g. 'https://www.mrlc.gov/geoserver/mrlc_download/nlcd_tcc_conus_2021_v2021-4/wcs?'
    ply = (geoseries or geodataframe) of the study area
    service_name = (string) name of the service e.g. mrlc_download__nlcd_tcc_conus_2021_v2021-4
    out_prefix = (string) prefix used to save each image

    returns a Raster object
    '''
    wcs=WebCoverageService(url)
    tcc=wcs.contents[service_name]
    bbox=tuple(ply.total_bounds)
    subsets=[('X',bbox[0],bbox[2]),('Y',bbox[1],bbox[3])]
    rsp=wcs.getCoverage(identifier=[tcc.id],subsets=subsets,format='geotiff')
    outpath='./'+out_prefix+'.tif'
    with open(outpath,'wb') as file:
        file.write(rsp.read())

    return Raster(outpath)


##### Step 2: Download the data

In [ ]:
# #Get Landsat data from STAC
outpath='ls82016.tif'
fnd=True
if(not os.path.exists(outpath)): #if the 2016 Landsat file does not exits, download it
    ls_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
    ls_name = "landsat-c2-l2"
    bnds=['red', 'blue', 'green', 'nir08', 'lwir11','swir16', 'swir22']
    ck=get_stac_data(nfp,ls_url,
                    name=ls_name,bands=bnds,
                    res=30,crs=5070,out_prefix='ls82016',datetime='2016-06-01/2016-06-30', #date time is very important here
                    query={'eo:cloud_cover':{'lt':10},'platform':{'eq':'landsat-8'}},
                    limit=1000)
    if(ck is None): fnd=False

if fnd: ls82016=Raster(outpath)
else: "No objects found in the query"

# #Get 2016 TCC data from Image Service
outpath='tcc20161.tif'
if(not os.path.exists(outpath)): #if the 2016 tree canopy cover file does not exits, download it
    url=r'https://apps.fs.usda.gov/fsgisx01/rest/services/RDW_LandscapeAndWildlife/NLCD_2016_TreeCanopyCover_CONUS/ImageServer'
    im_lst=get_image_service_data(url=url,ply=nfp,out_prefix='tcc2016',res=30,outSR=5070)

tcc2016=Raster(outpath) #should only be one tile


#Get elevation data from USFS 3Dep program web service
outpath='./dem.tif' #specify the output path
if(not os.path.exists(outpath)): #if the dem file does not exist, download it
    geo = nf.envelope.geometry[0] #get the geometry for extent of the study area
    d1=py3dep.get_dem(geo,30,4326).expand_dims({'band':1}) # use py3dep to get a xarray surface
    d2=Raster(d1).reproject(nfp.crs) #convert d1 into a Raster object
    d2.save(outpath) #save it to disk

dem=Raster(outpath) #load your dem from disk

#Don't we also need Landsat imagery for the year 2021? How can we get that?
# outpath='ls82021.tif'
# fnd=True
# if(not os.path.exists(outpath)): #if the 2016 Landsat file does not exits, download it
#     ls_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
#     ls_name = "landsat-c2-l2"
#     bnds=['red', 'blue', 'green', 'nir08', 'lwir11','swir16', 'swir22']
#     ck=get_stac_data(nfp,ls_url,
#                     name=ls_name,bands=bnds,
#                     res=30,crs=5070,out_prefix='ls82016',datetime='2021-06-01/2021-06-30', #date time is very important here
#                     query={'eo:cloud_cover':{'lt':10},'platform':{'eq':'landsat-8'}},
#                     limit=1000)
#     if(ck is None): fnd=False

# if fnd: ls82016=Raster(outpath)
# else: "No objects found in the query"

#TCC for 2021 does not exist on Landfire's image service. How can we get TCC for 2021?
# outpath='tcc2021.tif'
# if(not os.path.exists(outpath)): #if the 2021 tree canopy cover file does not exits, download it
#     url=r'https://www.mrlc.gov/geoserver/mrlc_download/nlcd_tcc_conus_2021_v2021-4/wcs?'
#     sn='mrlc_download__nlcd_tcc_conus_2021_v2021-4'
#     get_wcs_data(url=url,ply=nfp,service_name=sn,out_prefix='tcc2021')

# tcc2021=Raster(outpath) #should only be one tile



##### Step3: Visualize the Rasters

In [ ]:
#Use Raster Tools to display a web map of the first band in the Landsat image
display(ls82016.xdata)
display(ls82016.explore(band=1,cmap='Reds'))

#How can we do the same thing for our tree canopy cover and dem surfaces?
# display(tcc2016.xdata)
# display(tcc2016.explore(band=1,cmap='Greens'))
# display(dem.xdata)
# dem.explore(band=1,cmap='terrain')

#Can we visualize the Landsat image as a 3 band color composite?

### Section 4: Creating the Data Frame
#### In this section we will focus on creating a data frame for further analyses.

#### Key learning points:
- The utility of data frame
- How to use both Raster and Vector data
- How to visualize the raw data
- How to store the data

#### Coding Steps:
1. Extract spectral, elevation, and tcc values for each point
2. Merge data with points
3. Look at the data


##### Step 1: Extract the data

In [ ]:
lstbl=zonal.extract_points_eager(pnts,ls82016,'ls',axis=1).compute()
eltbl=zonal.extract_points_eager(pnts,dem,'el',axis=1).compute()
tcctbl=zonal.extract_points_eager(pnts,tcc2016,'tcc',axis=1).compute()

#why do we need to compute?

#what does axis=1 mean?


##### Step 2: Merge data with points

In [ ]:
atr=pd.concat([lstbl,eltbl,tcctbl],axis=1)
gdf=gpd.GeoDataFrame(atr,geometry=pnts)

#what does axis 1 mean in this context?

#Why do we want to use a geodataframe?

##### Step 3: Look at the data frame

In [ ]:
display(gdf)
gdf.explore(column='tcc_1')

#How can we save our geodataframe into a format others can use?
# gdf.to_file('data.shp.zip') # a
# gdf.to_csv('data.csv')
#

### Section 5: Cleaning Data
#### In this section we will explore our data and look for problems that may need to be fixed before we proceed

#### Key learning points:
- Data are often messy
- Missing data can be troublesome to deal with
- Ways to identify issues with the data and fix those issues
- plotting and graphing

#### Coding Steps:
1. Finding incorrect values, missing data, and extreme values
2. Replacing values
3. Dropping records


##### Step 1: Finding incorrect values or missing data
Here we make a distinction between missing data (NA), incorrect values (typos, nodata values), and extreme values. Records with missing data or incorrect values need to be closely examined to see if they need to be removed or replaced, while records with extreme values may highlight unique instances in the data that warrant further investigation. While there are many approaches to cleaning data, here are few common techniques:
- Look at the data for obvious errors
- Find null values and replace them with the mean of the feature or impute the closest value
- Find records with null values and drop those records from the dataset
- Use percentiles to identify extreme and potentially incorrect values
- Look at correlation


In [ ]:
#Create a map of locations overlaid on raster surfaces
import folium
m = nfp.explore(name='Boundary')
m = gdf.explore(m=m, color='yellow',name='Points')

folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Imagery',
        overlay = False,
        control = True
       ).add_to(m)


# comment and uncomment image layers and rerun
m = tcc2016.explore(band=1,map=m,cmap='Greens',name='TCC')
#m = ls82016.explore(band=1,map=m,cmap='Reds',name='Landsat Band 1')
#m = dem.explore(band=1,map=m,cmap='terrain',name='DEM')

m

#How can we add all three images to the same map?

#Do we see any obvious points we should exclude from our analysis?

#What is our population and what does that mean with regards to inference?



In [ ]:
#search for null values
na_rows=gdf[gdf.isna().any(axis=1)]
print('number of rows with na =',na_rows.shape[0])

#search for empty values
emp_rows=gdf[(gdf=='').any(axis=1)]
print('number of rows that have empty values =',emp_rows.shape[0])

#what about zero values?
zero_rows=gdf[(gdf==0).any(axis=1)]
print('number of rows that have zero values =',emp_rows.shape[0])

# It does not look like we have any missing data. If we did, how could we remove records or populate missing values?

# what about using the feature average?

# what about imputation?



In [ ]:
# describe the data
print('Summary Statistics')
display(atr.describe())

# use quantiles to look at extreme values
extreme=atr.quantile([0.01,0.99])
ex=(atr<extreme.min()).sum(axis=1) + (atr>extreme.max()).sum(axis=1) #sum the number of extreme values in each row
gdf['extreme'] = ex #add that attribute to gdf

# calculate correlation
print('Correlation Matrix')
display(atr.corr())

# map extreme values
print('Map of extreme values (blue to red increased number of extreme values)')
display(gdf[ex>0].explore(column='extreme', cmap= 'RdBu_r'))

#create a scatter plot matrix of values that highlight the extreme values
print('Scatter Plot Matrix (blue to red increased number of extreme values)')
display(pd.plotting.scatter_matrix(atr,figsize=(15,15),c=ex,cmap='RdBu_r'))


# what relationships do you see?

# does there appear to be any observations we should exclude from our analysis?

# How can we determine which observations are influential or have leverage?

# What if we extracted larger areas than just the cell?

# How could we incorporate texture?

# what if we created surface derivatives of elevation?

##### Step 2: Replacing values
In this example there were no missing values to replace but what if there were? We will explore 2 options commonly used to replace missing data; 1) using the mean and 2) imputing missing data. For demonstration purposes we will add some missing data to our gdf dataframe and then show how to replace values with the feature's mean value and impute values using [scikit-learn's impute module](https://scikit-learn.org/stable/modules/impute.html).

In [ ]:
# Lets make some messy data called mdata using our oud attribute dataframe, extreme values (extreme>5), and a random generator
sdt=atr[ex>5]
for r in range(sdt.shape[0]):
    vls=sdt.iloc[r].copy()
    c=np.random.randint(1,7,1)[0]
    vls.iloc[c]=np.nan
    sdt.iloc[r]=vls

sdt
mdata=atr.copy()
mdata[ex>5]=sdt

print('Number of extreme values = ',(ex>5).sum())
print('Number of nans =',mdata.isna().sum().sum())

#show the messy data
print('Here are the rows with nans...')
display(mdata[ex>5])


In [ ]:
#let's us scikit-learn's SimpleImputer to fill in the mean value for a nans
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean') #Specify the SimpleImputer (mean)
imp.fit(mdata) #fit the mdata for all non nan values
cdata=pd.DataFrame(imp.transform(mdata),columns=mdata.columns) #create the new data frame with no nans

#look at the changed records
cdata[ex>5]

In [ ]:
#let's us scikit-learn's IterativeImputer to fill in the nan values
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(mdata)
cdata2=pd.DataFrame(imp.transform(mdata),columns=mdata.columns) #create the new data frame with no nans

#look at the changed records
cdata2[ex>5]

#why are the values different?

#which technique is better?

##### Step 3: Dropping records
What if so much of the data for a record is gone or is somehow incorrect that it really does not add anything to our analysis or worse yet detracts from our analysis? In that case we may want to think about removing that record from our dataset. While this approach should be used as a last resort, it is very easy to implement. For demonstration purposes, we will show how to select records and remove them from our geodataframe.

In [ ]:
#let's drop all records with a nan value
cdata3=mdata.dropna()

print('The number of rows in cdata4 =',cdata3.shape[0])

#why do we need to remove na values?

#why is droping rows used as last resort?

#How can you change a know value that is incorrect?

#### Preprocessing
##### In this section we will begin looking at ways to address trending in our data and potentially reducing dimensionality.

##### Key learning points:
- The curse of dimensionality
- Transformations
- Ordination
- Deciding on predictors
- Using plotting and graphing

#### Building a predictive model
##### In this section we will explore how to build a model that can be used later to predict new estimates

##### Key learning points:
- modeling assumptions
- Bias
- training, testing, and validation
- Model estimates and error
- Estimation domain

#### Estimating % Tree Canopy Cover
##### In this section we will explore how to use a model to create new estimates and surfaces

##### Key learning points:
- Estimates may not reflect reality
- The power of more estimates
- Aggregating values
- Estimates of error